In [194]:
%matplotlib inline
%matplotlib nbagg
import pickle
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import prettytensor as pt

In [195]:
rows, cols = 70, 100
img_shape = (rows, cols)
img_flat_size = rows * cols
img_classes = 2
img_channels = 3

phase = pt.Phase.train

In [196]:
resized_folder_path = "/home/openroot/Tanmoy/Working Stuffs/myStuffs/havss-tf/ISIC-2017/resized/"

In [197]:
isic_pickle_file = resized_folder_path + "ISIC.pickle"
with open(isic_pickle_file, "rb") as f:
    save = pickle.load(f)
    train_dataset =  save["train_dataset"]
    train_labels =  save["train_labels"]
#     validation_dataset =  save["validation_dataset"]
#     validation_labels =  save["validation_labels"]
    test_dataset =  save["test_dataset"]
    test_labels =  save["test_labels"]
    del save
print("Training Set {0} {1}".format(train_dataset.shape, train_labels.shape))
#     print("Validation Set {0} {1}".format(validation_dataset.shape, validation_labels.shape))    
print("Test Set {0} {1}".format(test_dataset.shape, test_labels.shape))
print test_labels[0:10, :]
dataset_info = np.sum(test_labels[:, :], axis = 0)
print("Test Dataset Info : [1, 0] = {0}, [0, 1] = {1}".format(dataset_info[0], dataset_info[1]))
# dataset_info = np.sum(validation_labels[:, :], axis = 0)
# print("Validation Dataset Info : [1, 0] = {0}, [0, 1] = {1}".format(dataset_info[0], dataset_info[1]))
dataset_info = np.sum(train_labels[:, :], axis = 0)
print("Train Dataset Info : [1, 0] = {0}, [0, 1] = {1}".format(dataset_info[0], dataset_info[1]))
dataset_info = []

Training Set (37904, 70, 100, 3) (37904, 2)
Test Set (6152, 70, 100, 3) (6152, 2)
[[ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]]
Test Dataset Info : [1, 0] = 2786.0, [0, 1] = 3366.0
Train Dataset Info : [1, 0] = 20612.0, [0, 1] = 17292.0


In [198]:
tf.reset_default_graph() 

In [199]:
# input_x = tf.placeholder(tf.float32, [None, img_flat_size], name = "input_x") 
input_layer = tf.placeholder(tf.float32, [None, rows, cols, img_channels], name = "input_layer")
y_true = tf.placeholder(tf.float32, [None, img_classes], name = "y_true")
y_true_cls = tf.argmax(y_true, dimension = 1)

In [200]:
x_pretty = pt.wrap(input_layer)
with pt.defaults_scope(activation_fn = tf.nn.relu):
    y_pred, loss = x_pretty.\
        conv2d(kernel = 5, depth = 16, name = "conv_layer_1").\
        max_pool(kernel = 2, stride = 2).\
        dropout(keep_prob=0.8, phase=phase).\
        conv2d(kernel = 5, depth = 32, name = "conv_layer_2").\
        max_pool(kernel = 2, stride = 2).\
        dropout(keep_prob=0.8, phase=phase).\
        conv2d(kernel = 5, depth = 64, name = "conv_layer_3").\
        max_pool(kernel = 2, stride = 2).\
        flatten().\
        fully_connected(size = 256, name = "fc_layer_2", activation_fn = tf.nn.relu).\
        softmax_classifier(num_classes = img_classes, labels = y_true)

In [201]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(loss)

In [202]:
y_pred_cls = tf.argmax(y_pred, dimension = 1)

In [203]:
correct_pred = tf.equal(y_true_cls, y_pred_cls)

In [204]:
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [205]:
train_batch_size = 32
test_batch_size = 10
prev_train_acc = 0
prev_train_acc_step = 0
min_train_acc_step = 1000
offset = 0

In [206]:
# num_steps = 12000
# with tf.Session() as sess:
#     tf.global_variables_initializer().run()
#     print("Variable Initialized")
#     for step in range(num_steps):
#         offset = (step * train_batch_size) % (train_labels.shape[0] - train_batch_size)
# #         print offset
#         batch_train = train_dataset[offset:(offset + train_batch_size), :, : ,:]
#         batch_labels = train_labels[offset:(offset + train_batch_size), :]
#         feed_dict_train = {
#             input_layer: batch_train,
#             y_true: batch_labels
#         }
        
#         _, l = sess.run([optimizer, loss], feed_dict = feed_dict_train)
#         print("Step : {0} - Loss : {1}".format(step, l))
        

#         if step % 200 :
#             feed_dict_valid = {
#                 input_layer: validation_dataset,
#                 y_true: validation_labels
#             }

#             valid_acc = sess.run([accuracy], feed_dict = feed_dict_valid)
#             print("Step : {0} - Validation Accuracy : {1} ".format(step, train_loss))
            
#     phase = pt.Phase.test    
#     feed_dict_test = {
#         input_layer: test_dataset,
#         y_true: test_labels
#     }
#     true_value_class, acc, pred_value, pred_value_class = sess.run([y_true_cls, accuracy, y_pred, y_pred_cls], feed_dict = feed_dict_test)
#     print("Accuracy {:.2%}".format(acc))
#     print("True Value Class")
#     print true_value_class[0:10]
#     print("Pred Value")
#     print pred_value[0:20]
#     print("Pred Value Class")
#     print pred_value_class

In [207]:
init_op = tf.global_variables_initializer()
saver = tf.train.Saver()

In [208]:
def runModel(epoch):
    with tf.Session() as sess:
        global train_labels, train_dataset
        sess.run(init_op)
        print("Variables Initialized")
        total_data_len = train_labels.shape[0]
        print total_data_len
        num_step = int(total_data_len / train_batch_size)
        print("Batch Size : {0}".format(train_batch_size))
        print("Starting Training")
#         for i in range(epoch):
#             print("Epoch - {0} ".format(i))
#             for step in range(num_step):
#                 offset = (step * train_batch_size) % (total_data_len - train_batch_size)
#                 batch_train = train_dataset[offset:(offset + train_batch_size), :, :, :]
#                 batch_labels = train_labels[offset:(offset + train_batch_size), :]
#                 feed_dict_train = {
#                     input_layer: batch_train,
#                     y_true: batch_labels
#                 }

#                 _, train_loss = sess.run([optimizer, loss], feed_dict = feed_dict_train)
#                 print("Step : {0} - Training Loss : {1}".format(step, train_loss))
                
#                 if step % 200 :
#                     feed_dict_valid = {
#                         input_layer: validation_dataset[:2000,:,:,:],
#                         y_true: validation_labels[:2000,:]
#                     }
                    
#                     valid_acc = sess.run([accuracy], feed_dict = feed_dict_valid)
#                     print("Step : {0} - Validation Accuracy : {1} ".format(step, train_loss))
                    
#                     if valid_acc > prev_train_acc:
#                         print("Imporvement Found : {0}, Step Difference : {1}".format(valid_acc - prev_train_acc, step - prev_train_acc_step))
#                         prev_train_acc_step = step
#                         prev_train_acc = valid_acc
#                     else:
#                         if prev_train_acc_step - step > min_train_acc_step:
#                             print("No Improvement Found : Step Difference : {0}".format(step - prev_train_acc_step))
#                             print("Therefore Ending Training")
#                             break

        save_path = saver.save(sess, "/home/openroot/tensorboard/isic/model")
        print("Model saved in file: %s" % save_path)
        saver.export_meta_graph('/home/openroot/tensorboard/isic/model.meta')
        train_dataset = []
        train_labels = []
        
        print("Training Completed!!!")
        print("Staring Testing Phase")
        phase = pt.Phase.test    
        
        total_test_data_len = test_labels.shape[0]
        final_test_pred = np.array([])
        print total_test_data_len
        i = 0
        while (i < total_test_data_len - test_batch_size):    
            print("i - {0} to {1}".format(i, i + test_batch_size))
            feed_dict_test = {
                input_layer: test_dataset[i:(i + test_batch_size),:,:,:],
                y_true: test_labels[i:(i + test_batch_size),:]
            }      
            test_batch_pred = sess.run(y_pred, feed_dict = feed_dict_test)
#             if i % 200:
#                 print("Training Step : {0}, Pred : {1}, Shape : {2}". format(i, test_batch_pred, test_batch_pred.shape))
#                 print test_batch_pred
#                 print test_batch_pred.shape
            final_test_pred = np.append(final_test_pred, test_batch_pred)
            i = i + test_batch_size
        
        print final_test_pred.shape
        print "Value of i - {0}".format(i)
        feed_dict_test = {
            input_layer: test_dataset[i:total_test_data_len,:,:,:],
            y_true: test_labels[i:total_test_data_len,:]
        }      
        test_batch_pred = sess.run(y_pred, feed_dict = feed_dict_test)
        final_test_pred = np.append(final_test_pred, test_batch_pred)
        print final_test_pred.shape   
    
            
        print "------------ Shape"
    
        final_test_pred = np.reshape(final_test_pred, (-1, 2))
        print final_test_pred.shape
        print final_test_pred[0:15,:]
        
        print test_labels[0:15,:]
        
        print "------------ Pred Test"
        
        final_test_pred_cls = np.argmax(final_test_pred, axis = 1)
        print final_test_pred_cls.shape
        print final_test_pred_cls[0:10]
        
        print "------------ True Test"
        
        final_test_true_cls = np.argmax(test_labels, axis = 1)
        print final_test_true_cls.shape
        print final_test_true_cls[0:10]   
        
        print "------------ Acc Test"
        final_correct_test = np.equal(final_test_pred_cls, final_test_true_cls).astype(np.float32)
        print final_correct_test[0:10]
        test_acc = np.mean(final_correct_test, dtype=np.float64)
        print "Test Accuracy : {0}".format(test_acc)
        

In [209]:
runModel(10)

Variables Initialized
37904
Batch Size : 32
Starting Training
Model saved in file: /home/openroot/tensorboard/isic/model
Training Completed!!!
Staring Testing Phase
6152
i - 0 to 10
i - 10 to 20
i - 20 to 30
i - 30 to 40
i - 40 to 50
i - 50 to 60
i - 60 to 70
i - 70 to 80
i - 80 to 90
i - 90 to 100
i - 100 to 110
i - 110 to 120
i - 120 to 130
i - 130 to 140
i - 140 to 150
i - 150 to 160
i - 160 to 170
i - 170 to 180
i - 180 to 190
i - 190 to 200
i - 200 to 210
i - 210 to 220
i - 220 to 230
i - 230 to 240
i - 240 to 250
i - 250 to 260
i - 260 to 270
i - 270 to 280
i - 280 to 290
i - 290 to 300
i - 300 to 310
i - 310 to 320
i - 320 to 330
i - 330 to 340
i - 340 to 350
i - 350 to 360
i - 360 to 370
i - 370 to 380
i - 380 to 390
i - 390 to 400
i - 400 to 410
i - 410 to 420
i - 420 to 430
i - 430 to 440
i - 440 to 450
i - 450 to 460
i - 460 to 470
i - 470 to 480
i - 480 to 490
i - 490 to 500
i - 500 to 510
i - 510 to 520
i - 520 to 530
i - 530 to 540
i - 540 to 550
i - 550 to 560
i - 560 to

i - 4870 to 4880
i - 4880 to 4890
i - 4890 to 4900
i - 4900 to 4910
i - 4910 to 4920
i - 4920 to 4930
i - 4930 to 4940
i - 4940 to 4950
i - 4950 to 4960
i - 4960 to 4970
i - 4970 to 4980
i - 4980 to 4990
i - 4990 to 5000
i - 5000 to 5010
i - 5010 to 5020
i - 5020 to 5030
i - 5030 to 5040
i - 5040 to 5050
i - 5050 to 5060
i - 5060 to 5070
i - 5070 to 5080
i - 5080 to 5090
i - 5090 to 5100
i - 5100 to 5110
i - 5110 to 5120
i - 5120 to 5130
i - 5130 to 5140
i - 5140 to 5150
i - 5150 to 5160
i - 5160 to 5170
i - 5170 to 5180
i - 5180 to 5190
i - 5190 to 5200
i - 5200 to 5210
i - 5210 to 5220
i - 5220 to 5230
i - 5230 to 5240
i - 5240 to 5250
i - 5250 to 5260
i - 5260 to 5270
i - 5270 to 5280
i - 5280 to 5290
i - 5290 to 5300
i - 5300 to 5310
i - 5310 to 5320
i - 5320 to 5330
i - 5330 to 5340
i - 5340 to 5350
i - 5350 to 5360
i - 5360 to 5370
i - 5370 to 5380
i - 5380 to 5390
i - 5390 to 5400
i - 5400 to 5410
i - 5410 to 5420
i - 5420 to 5430
i - 5430 to 5440
i - 5440 to 5450
i - 5450 to 54